# Part 1 - Create Toronto Neighborhood Dataframe

### Import libraries
Import the libraries required to read the web page and convert the table to dataframe

In [1]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup

### Read the web page
Read the table in the wikipedia page using Beautiful Soup library

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request = rq.get(url)
soup = BeautifulSoup(request.content,'lxml')
postal_table = soup.find('table',class_='wikitable sortable')

### Convert the soup data into daraframe

In [3]:
# Read the HTML as a dataframe
postal_df = pd.read_html(str(postal_table))[0]

# Set the column names as required
postal_df.columns = ['PostalCode','Borough','Neighborhood']

# Remove the first row which is same as the column header
postal_df.drop([0], axis=0, inplace=True)

# Get the index list with Borough as 'Not assigned' and drop it from the dataframe
NA_index = postal_df[postal_df['Borough']=='Not assigned'].index.tolist()
postal_df.drop(NA_index, axis=0, inplace=True)

In [4]:
# Replace '/' with comma for the rows with multiple neighborhood data
postal_df['Neighborhood'] = postal_df['Neighborhood'].str.replace(' /',',')
postal_df['Neighborhood'] = postal_df['Neighborhood'].str.replace('/',',')

In [5]:
#Checking for any NaN or Not Assigned data in Neighborhood
postal_df[postal_df['Neighborhood'].isin(['Not assigned',np.nan])]

,PostalCode,Borough,Neighborhood


As there are no rows with NaN or Not assigned data in Neighborhood, the dataframe looks good for further processing

In [6]:
# Reset the index after all processing
postal_df.reset_index(drop=True, inplace=True)

### Final shape of the dataframe

In [9]:
print(postal_df.shape)
postal_df.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Part 2 - Add the Latitude and Longitude to Dataframe

### Install and Import libraries for Geocoder

In [ ]:
#!pip install geocoder

In [10]:
import geocoder

### Get the latitude and longitude using Geocoder

In [11]:
#Function to call the geocder iteratively to get the latitude and longitude until values are returned
def getCoords(row):
    print(row['PostalCode'] + ', Toronto, ON')
    coords = None
    i = 1
    while (coords is None):
        print('Attempt ' + str(i))
        geo = geocoder.google('{}, Toronto, Ontario'.format(row['PostalCode']))
        coords = geo.lating
        i += 1
    return(coords)

In [12]:
#Read each row from the dataframe and call the getCoords function to get the latitide and Longitude
for i, row in postal_df.iterrows():
    lat_long = getCoords(row)
    row['Latitude'] = lat_long[0]
    row['Longitude'] = lat_long[1]
    print(row.iloc[0:i+1,:])

M3A, Toronto, ON
Attempt 1
Attempt 2
Attempt 3
Attempt 4
Attempt 5
Attempt 6
Attempt 7
Attempt 8
Attempt 9


KeyboardInterrupt: 

The geocoder API didn't work in returning the coordinates. So I am using the CSV file provided in the assignment to add the latitude and longitude

### Get Latitude and Longitude from the CSV file

In [13]:
#Read the csv to create the dataframe
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
geo_df.shape

(103, 3)

In [15]:
#Merge the dataframes to add the latitude and longitude to the neighborhood data
postal_df = postal_df.merge(geo_df, how='left', left_on='PostalCode', right_on='Postal Code')
postal_df.drop('Postal Code', axis=1, inplace=True)
postal_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [16]:
postal_df.shape

(103, 5)

# Part 3 - Exploring and Clustering the Toronto Neighborhood

### Import Libraries

In [17]:
import json 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

### Explore the Neighborhood data

In [18]:
#Get the number of Boroughs and Neighborhood
print('Number of Boroughs: ' + str(len(set(postal_df['Borough']))))
print('Number of Neighborhood: ' + str(postal_df.shape[0]))

Number of Boroughs: 10
Number of Neighborhood: 103


In [19]:
#Get the number of Neighborhood for each Borough
postal_df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

**Get the coordinates of Toronto, ON**

In [20]:
city = 'Toronto, ON'
geolocator = Nominatim(user_agent="foursquare_agent")
loc = geolocator.geocode(city)
latitude = loc.latitude
longitude = loc.longitude
print('The geograpical coordinate of Toronto, ON are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON are 43.6534817, -79.3839347.


**Plot the neighborhood locations in the Toronto, ON map**

In [21]:
#Prepare the map focused on Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add the marker for the neighborhood in Toronto city
for lat, long, name in zip(postal_df['Latitude'], postal_df['Longitude'], postal_df['Neighborhood']):
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker([lat, long],
                        radius=3,
                        popup=label,
                        color='magenta',). add_to(map_toronto)

#Display the Toronto map with neighborhood markers
map_toronto

**Initialize the parameters for the Foursquare API**

In [22]:
CLIENT_ID = '1WSXSUNXPFJ2BHBIOBXKVY2HKQK1HKEKQBBOGDUSR1PPPQR5' 
CLIENT_SECRET = 'XZWPMWPR2P2RWXXW35XBBLYGNZ4T2WA2CE1HUJQXATFG31EJ' 
VERSION = '20180605' 

**Create a function to explore all the venues for each neighborhoods in Toronto**

In [23]:
def getVenues(latitude, longitude, neighborhood):
    
    #Initialize the basic parameters
    radius = 500
    limit=100
    venues = []
    
    #Process each neighborhood in a loop
    for lat, long, name in zip(latitude, longitude, neighborhood):
        
        #Form the explore url to make the API call
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius,
        limit)
        
        #Invoke the API and save the result
        results = rq.get(url).json()
        res_item = results['response']['groups'][0]['items']
        
        #Process the result and form a list of tuples
        for item in res_item:
            venue_name = item['venue']['name']
            venue_lat = item['venue']['location']['lat']
            venue_long = item['venue']['location']['lng']
            venue_cat = item['venue']['categories'][0]['name']
            venues.append([(name, lat, long, venue_name, venue_lat, venue_long, venue_cat)])
            
            #Convert the list of tuples into a dataframe and give meaningful column names
            venues_df = pd.DataFrame(item[0] for item in venues)
            venues_df.columns = ['Neighborhood Name', 'Latitude', 'Longitude', ' Venue', 'Venue Latitude', 'Venue Longitude', 'Category']
    
    #Return the dataframe of neighborhood venues
    return(venues_df)

**Call the getVenues function to get the near by venues for all the neighborhood**

In [24]:
toronto_venues = getVenues(postal_df['Latitude'], postal_df['Longitude'], postal_df['Neighborhood'])

**Verify the shape of the resulting dataframe and data by using head()**

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(2139, 7)


,Neighborhood Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


**Check the number of venues for each neighborhood**

In [26]:
toronto_venues.groupby('Neighborhood Name').count()[['Category']]

,Category
Neighborhood Name,
Agincourt,4
"Alderwood, Long Branch",9
"Bathurst Manor, Wilson Heights, Downsview North",20
Bayview Village,4
"Bedford Park, Lawrence Manor East",24
Berczy Park,59
"Birch Cliff, Cliffside West",4
"Brockton, Parkdale Village, Exhibition Place",25
Business reply mail Processing CentrE,18


**Check the list of unique categories**

In [27]:
len(set(toronto_venues['Category']))

267

**Change the categories to columns to analyze the neighborhood**

In [28]:
#Convert list of unique categories to columns
venue_categorized = pd.get_dummies(toronto_venues['Category'])

#Add the Nrighborhood names to the category dataframe
venue_categorized = pd.concat([toronto_venues[['Neighborhood Name']], venue_categorized], axis=1)
print(venue_categorized.shape)
venue_categorized.head()

(2139, 268)


,Neighborhood Name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Group by neighborhood and get the mean for each category to know the frequency of each category**

In [29]:
neigh_group = venue_categorized.groupby('Neighborhood Name').mean().reset_index()
print(neigh_group.shape)
neigh_group.head()

(94, 268)


,Neighborhood Name,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Get the most common venues for each neighborhood**

In [30]:
#Let's get top 10 venues
top_count = 10

#Set the column names for the top 10 venue dataframe
columns = ['Neighborhood Name']
for ind in np.arange(top_count):
    columns.append('Common Venue #'+str(ind+1))
common_venues = pd.DataFrame(columns=columns)

#Insert the common venues data in the dataframe
common_venues['Neighborhood Name'] = neigh_group['Neighborhood Name']
for ind in np.arange(neigh_group.shape[0]):
    common_venues.iloc[ind, 1:] = neigh_group.iloc[ind, 1:].sort_values(ascending=False).index.values[0:top_count]

print(common_venues.shape)
common_venues.head()

(94, 11)


,Neighborhood Name,Common Venue #1,Common Venue #2,Common Venue #3,Common Venue #4,Common Venue #5,Common Venue #6,Common Venue #7,Common Venue #8,Common Venue #9,Common Venue #10
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pool,Skating Rink,Gym,Pharmacy,Coffee Shop,Pub,Sandwich Place,Yoga Studio,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Shopping Mall,Pizza Place,Sushi Restaurant,Middle Eastern Restaurant,Deli / Bodega,Diner,Ice Cream Shop,Fried Chicken Joint
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Restaurant,Italian Restaurant,Coffee Shop,Greek Restaurant,Pizza Place,Breakfast Spot,Juice Bar,Butcher,Café


## Cluster the neighborhood data using K-Means

In [31]:
#Set the number of clusters to 3
clusters = 3

#Dataframe for modeling
neigh_cluster = neigh_group.iloc[:, 1:]

#Define KMeans and fit the model
kmeans = KMeans(init='k-means++', n_clusters=clusters, n_init=12).fit(neigh_cluster)

print(set(kmeans.labels_))
print(len(kmeans.labels_))
kmeans.labels_[0:10]

{0, 1, 2}
94


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

**Add the cluster label to the common_venues dataframe**

In [32]:
common_venues['Cluster'] = kmeans.labels_

In [33]:
common_venues.head()

,Neighborhood Name,Common Venue #1,Common Venue #2,Common Venue #3,Common Venue #4,Common Venue #5,Common Venue #6,Common Venue #7,Common Venue #8,Common Venue #9,Common Venue #10,Cluster
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,0
1,"Alderwood, Long Branch",Pizza Place,Pool,Skating Rink,Gym,Pharmacy,Coffee Shop,Pub,Sandwich Place,Yoga Studio,Dim Sum Restaurant,0
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Shopping Mall,Pizza Place,Sushi Restaurant,Middle Eastern Restaurant,Deli / Bodega,Diner,Ice Cream Shop,Fried Chicken Joint,0
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,0
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Restaurant,Italian Restaurant,Coffee Shop,Greek Restaurant,Pizza Place,Breakfast Spot,Juice Bar,Butcher,Café,0


**Merge the common venues data to the Toronto neighborhood base data**

In [34]:
toronto_neigh_venues = postal_df.merge(common_venues, left_on='Neighborhood', right_on='Neighborhood Name').drop('Neighborhood Name', axis=1)
toronto_neigh_venues.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Common Venue #1,Common Venue #2,Common Venue #3,Common Venue #4,Common Venue #5,Common Venue #6,Common Venue #7,Common Venue #8,Common Venue #9,Common Venue #10,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,Park,Food & Drink Shop,Pool,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Falafel Restaurant,2
1,M4A,North York,Victoria Village,43.725882,-79.315572,Portuguese Restaurant,French Restaurant,Hockey Arena,Coffee Shop,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Coffee Shop,Park,Pub,Bakery,Café,Breakfast Spot,Theater,Mexican Restaurant,Bank,Ice Cream Shop,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Furniture / Home Store,Clothing Store,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Arts & Crafts Store,Accessories Store,Event Space,Women's Store,Coffee Shop,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Coffee Shop,Sushi Restaurant,Diner,Creperie,Bar,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café,0


**Plot the map with the cluster of neighborhoods**

In [35]:
#Get the latitude and Longitude for Toronto
city = 'Toronto, ON'
geolocator = Nominatim(user_agent="foursquare_agent")
loc = geolocator.geocode(city)
latitude = loc.latitude
longitude = loc.longitude

#Prepare the map focused on Toronto
map_cluster = folium.Map(location=[latitude, longitude], zoom_start=11)

#Set colors for clusters
colors_array = cm.rainbow(np.linspace(0, 1, clusters))
color_list = [colors.rgb2hex(i) for i in colors_array]

# Add the marker for the neighborhood in Toronto city
for lat, long, name, cluster in zip(toronto_neigh_venues['Latitude'], toronto_neigh_venues['Longitude'], toronto_neigh_venues['Neighborhood'], toronto_neigh_venues['Cluster']):
    label = folium.Popup(name+' - Cluster '+str(cluster), parse_html=True)
    folium.CircleMarker([lat, long],
                        radius=3,
                        popup=label,
                        color=color_list[cluster]). add_to(map_cluster)

#Display the Toronto map with neighborhood markers
map_cluster

In [36]:
toronto_neigh_venues[toronto_neigh_venues['Cluster']==0].drop(['PostalCode', 'Borough', 'Latitude', 'Longitude','Cluster'], axis=1)

,Neighborhood,Common Venue #1,Common Venue #2,Common Venue #3,Common Venue #4,Common Venue #5,Common Venue #6,Common Venue #7,Common Venue #8,Common Venue #9,Common Venue #10
1,Victoria Village,Portuguese Restaurant,French Restaurant,Hockey Arena,Coffee Shop,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,"Regent Park, Harbourfront",Coffee Shop,Park,Pub,Bakery,Café,Breakfast Spot,Theater,Mexican Restaurant,Bank,Ice Cream Shop
3,"Lawrence Manor, Lawrence Heights",Furniture / Home Store,Clothing Store,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Arts & Crafts Store,Accessories Store,Event Space,Women's Store,Coffee Shop
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Diner,Creperie,Bar,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café
5,"Malvern, Rouge",Fast Food Restaurant,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
6,Don Mills,Beer Store,Restaurant,Gym,Coffee Shop,Japanese Restaurant,Asian Restaurant,Athletics & Sports,Supermarket,Café,Caribbean Restaurant
7,Don Mills,Beer Store,Restaurant,Gym,Coffee Shop,Japanese Restaurant,Asian Restaurant,Athletics & Sports,Supermarket,Café,Caribbean Restaurant
8,"Parkview Hill, Woodbine Gardens",Pizza Place,Pet Store,Gastropub,Café,Bank,Intersection,Athletics & Sports,Fast Food Restaurant,Gym / Fitness Center,Pharmacy
9,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Japanese Restaurant,Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Theater,Hotel
10,Glencairn,Park,Japanese Restaurant,Pizza Place,Pub,Metro Station,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant


In [37]:
toronto_neigh_venues[toronto_neigh_venues['Cluster']==1].drop(['PostalCode', 'Borough', 'Latitude', 'Longitude','Cluster'], axis=1)

,Neighborhood,Common Venue #1,Common Venue #2,Common Venue #3,Common Venue #4,Common Venue #5,Common Venue #6,Common Venue #7,Common Venue #8,Common Venue #9,Common Venue #10
51,"Willowdale, Newtonbrook",Piano Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [38]:
toronto_neigh_venues[toronto_neigh_venues['Cluster']==2].drop(['PostalCode', 'Borough', 'Latitude', 'Longitude','Cluster'], axis=1)

,Neighborhood,Common Venue #1,Common Venue #2,Common Venue #3,Common Venue #4,Common Venue #5,Common Venue #6,Common Venue #7,Common Venue #8,Common Venue #9,Common Venue #10
0,Parkwoods,Park,Food & Drink Shop,Pool,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Falafel Restaurant
19,Caledonia-Fairbanks,Park,Pool,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
33,East Toronto,Metro Station,Park,Convenience Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
48,"North Park, Maple Leaf Park, Upwood Park",Park,Construction & Landscaping,Bakery,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
59,Lawrence Park,Park,Swim School,Bus Line,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Fast Food Restaurant
62,Weston,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
64,York Mills West,Park,Convenience Store,Bank,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
66,Forest Hill North & West,Jewelry Store,Park,Sushi Restaurant,Trail,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
80,"Moore Park, Summerhill East",Tennis Court,Restaurant,Park,Concert Hall,Construction & Landscaping,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
82,"Milliken, Agincourt North, Steeles East, L'Amo...",Park,Gym,Playground,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
